# Feature Engineering Script

**After Second Bus (SB) Data, waze data by the same street and default waze data**

Input variables: 89 + 1 (id)

In [1]:
# importing required packages
import pandas as pd
# import pandas_profiling
import numpy as np
import os
import featuretools as ft

In [2]:
# Choose the city of analysis

# city = 'Recife'
# city = 'CG'
city = 'Curitiba'

In [3]:
# Put the integrated data path

integrated_data_path = os.getcwd() + "/../data/output/" + city + "/"

In [6]:
# Importing the data

output_files = []
days_of_analysis = 0

for dir_name in os.listdir(integrated_data_path):
    
    if dir_name.startswith("output_"): #to get the dir per day
        dir_path = integrated_data_path + dir_name
        days_of_analysis = days_of_analysis+1
        
        for file_name in os.listdir(dir_path):
            
            if file_name.startswith("part-"):
                file_path = dir_path + "/" + file_name
                
                # Join multiple files of the same day
                df = pd.read_csv(file_path, index_col=None, header=0)
                output_files.append(df)
                
print('Days of analysis: ', days_of_analysis)

# Join all output files            
all_integrated_data = pd.concat(output_files, axis=0, ignore_index=True)

all_integrated_data.head(5)

Days of analysis:  1


,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,alertIsJamUnifiedAlertSB,alertInScaleSB,jamUpdateDateTimeSB,jamExpirationDateTimeSB,jamBlockTypeSB,jamDelaySB,jamLengthSB,jamLevelSB,jamSeveritySB,jamSpeedKMSB
0,304,1,2620,high_frequency,6614952,-25.435736,-49.30742,15614.581,DL301,1639934.0,...,False,False,2019-01-02 06:58:07,NaN,NORMAL,0,0,0,0,NaN
1,304,1,2620,high_frequency,6614952,-25.435736,-49.30742,15614.581,DL303,1645227.0,...,False,False,2019-01-02 07:14:44,NaN,NORMAL,0,0,0,0,NaN
2,304,1,2620,high_frequency,6614952,-25.435736,-49.30742,15614.581,LL305,4287173.0,...,False,False,2019-01-02 07:20:02,NaN,NORMAL,0,0,0,0,NaN
3,304,1,2620,high_frequency,6614952,-25.435736,-49.30742,15614.581,LL301,4280218.0,...,False,False,2019-01-02 07:26:44,NaN,NORMAL,0,0,0,0,NaN
4,304,1,2620,high_frequency,6614952,-25.435736,-49.30742,15614.581,LL308,4295954.0,...,False,False,2019-01-02 07:37:08,NaN,NORMAL,0,0,0,0,NaN


In [7]:
# Sorting by bus1-bus2-time

all_integrated_data.sort_values(by=['busCode', "busCodeSB", "gps_datetime"], inplace=True)
all_integrated_data.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,alertIsJamUnifiedAlertSB,alertInScaleSB,jamUpdateDateTimeSB,jamExpirationDateTimeSB,jamBlockTypeSB,jamDelaySB,jamLengthSB,jamLevelSB,jamSeveritySB,jamSpeedKMSB
1898,202,13,1875,high_frequency,5846943,-25.387823,-49.221917,6501.445,BD136,NaN,...,False,False,2019-01-02 19:56:03,NaN,NORMAL,0,0,0,0,NaN
1299,202,13,1875,high_frequency,5846958,-25.389683,-49.223023,6789.278,BD136,NaN,...,False,False,2019-01-02 19:59:53,NaN,NORMAL,0,0,0,0,NaN
3074,202,13,1875,high_frequency,5846970,-25.391065,-49.224296,7006.488,BD136,NaN,...,False,False,2019-01-02 19:59:56,NaN,NORMAL,0,0,0,0,NaN
2426,202,13,1875,high_frequency,5846977,-25.392437,-49.226385,7265.888,BD136,NaN,...,False,False,2019-01-02 20:00:00,NaN,NORMAL,0,0,0,0,NaN
1192,202,13,1875,high_frequency,5847061,-25.393398,-49.240938,9307.862,BD136,857475.0,...,False,False,2019-01-02 20:01:44,NaN,NORMAL,0,0,0,0,NaN


The following variables are objetc type and should be converted to the origin types because the correlation matrix.

In [6]:
all_integrated_data.dtypes

route                                   int64
tripNum                                 int64
shapeId                                 int64
routeFrequency                         object
shapeSequence                           int64
shapeLat                              float64
shapeLon                              float64
distanceTraveledShape                 float64
busCode                                object
gpsPointId                            float64
gpsLat                                float64
gpsLon                                float64
distanceToShapePoint                  float64
gps_datetime                           object
stopPointId                             int64
problem                                object
precipitation                         float64
precipitationTime                      object
precipitationStationDistance          float64
alertDateTime                          object
alertSubtype                           object
alertType                         

In [8]:
all_integrated_data['shapeId'] = all_integrated_data['shapeId'].astype(str)
all_integrated_data['routeFrequency'] = all_integrated_data['routeFrequency'].astype(str)
all_integrated_data['gps_datetime'] = all_integrated_data['gps_datetime'].astype('datetime64[ns]')
all_integrated_data['problem'] = all_integrated_data['problem'].astype(str)
all_integrated_data['precipitationTime'] = all_integrated_data['precipitationTime'].astype('datetime64[ns]')
all_integrated_data['alertDateTime'] = all_integrated_data['alertDateTime'].astype('datetime64[ns]')
all_integrated_data['alertSubtype'] = all_integrated_data['alertSubtype'].astype(str)
all_integrated_data['alertSubtype'].replace("nan", "-", inplace=True)
all_integrated_data['alertType'] = all_integrated_data['alertType'].astype(str)
all_integrated_data['alertType'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlert'] = all_integrated_data['alertIsJamUnifiedAlert'].astype(bool)
all_integrated_data['alertInScale'] = all_integrated_data['alertInScale'].astype(bool)
all_integrated_data['jamUpdateDateTime'] = all_integrated_data['jamUpdateDateTime'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTime'] = all_integrated_data['jamExpirationDateTime'].astype('datetime64[ns]')
all_integrated_data['jamBlockType'] = all_integrated_data['jamBlockType'].astype(str)
all_integrated_data['jamBlockType'].replace("nan", "-", inplace=True)

all_integrated_data['gps_datetimeSB'] = all_integrated_data['gps_datetimeSB'].astype('datetime64[ns]')
all_integrated_data['problemSB'] = all_integrated_data['problemSB'].astype(str)
all_integrated_data['precipitationTimeSB'] = all_integrated_data['precipitationTimeSB'].astype('datetime64[ns]')
all_integrated_data['alertDateTimeSB'] = all_integrated_data['alertDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['alertSubtypeSB'] = all_integrated_data['alertSubtypeSB'].astype(str)
all_integrated_data['alertSubtypeSB'].replace("nan", "-", inplace=True)
all_integrated_data['alertTypeSB'] = all_integrated_data['alertTypeSB'].astype(str)
all_integrated_data['alertTypeSB'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlertSB'] = all_integrated_data['alertIsJamUnifiedAlertSB'].astype(bool)
all_integrated_data['alertInScaleSB'] = all_integrated_data['alertInScaleSB'].astype(bool)
all_integrated_data['jamUpdateDateTimeSB'] = all_integrated_data['jamUpdateDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTimeSB'] = all_integrated_data['jamExpirationDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['jamBlockTypeSB'] = all_integrated_data['jamBlockTypeSB'].astype(str)
all_integrated_data['jamBlockTypeSB'].replace("nan", "-", inplace=True)

all_integrated_data.dtypes

route               int64
tripNum             int64
shapeId            object
routeFrequency     object
shapeSequence       int64
                   ...   
jamDelaySB          int64
jamLengthSB         int64
jamLevelSB          int64
jamSeveritySB       int64
jamSpeedKMSB      float64
Length: 89, dtype: object

In [8]:
# Descriptive statistics
profile = pandas_profiling.ProfileReport(all_integrated_data, check_correlation=True, correlation_threshold=0.5)

In [9]:
profile.to_file(outputfile=integrated_data_path + "output_profile_integrated_data.html")

In [11]:
all_integrated_data['jamExpirationDateTimeSB'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [9]:
# deleting jamExpirationDateTime because there is no value
all_integrated_data.drop('jamExpirationDateTime', axis=1, inplace=True)
all_integrated_data.drop('jamExpirationDateTimeSB', axis=1, inplace=True)

In [26]:
# saving integrated data before generate new feats
output_path = integrated_data_path + "/" + "integrated_data_before_new_feats.csv"
all_integrated_data.to_csv(output_path, index=False)

* Maybe it didn't measuring correlation because a lot of variables.

### 2. Creating new features


#### 2.1 With Feature Tools

In [10]:
# Creating an entity set
es = ft.EntitySet(id="all_data")

entities = es.entity_from_dataframe(entity_id="id", dataframe=all_integrated_data, make_index=True,
                                    index='id', time_index='gps_datetime')
entities

Entityset: all_data
  Entities:
    id [Rows: 3910, Columns: 88]
  Relationships:
    No relationships

In [11]:
entities.entities

[Entity: id
   Variables:
     id (dtype: index)
     route (dtype: numeric)
     tripNum (dtype: numeric)
     shapeId (dtype: categorical)
     routeFrequency (dtype: categorical)
     shapeSequence (dtype: numeric)
     shapeLat (dtype: numeric)
     shapeLon (dtype: numeric)
     distanceTraveledShape (dtype: numeric)
     busCode (dtype: categorical)
     gpsPointId (dtype: numeric)
     gpsLat (dtype: numeric)
     gpsLon (dtype: numeric)
     distanceToShapePoint (dtype: numeric)
     gps_datetime (dtype: datetime_time_index)
     stopPointId (dtype: numeric)
     problem (dtype: categorical)
     precipitation (dtype: numeric)
     precipitationTime (dtype: datetime)
     precipitationStationDistance (dtype: numeric)
     alertDateTime (dtype: datetime)
     alertSubtype (dtype: categorical)
     alertType (dtype: categorical)
     alertRoadType (dtype: numeric)
     alertConfidence (dtype: numeric)
     alertNComments (dtype: numeric)
     alertNImages (dtype: numeric)
     al

**Para a variável alvo (dados de ônibus por parada de ônibus), podemos gerar as seguintes variáveis:**

In [13]:
feature_matrix, features_defs = ft.dfs(entityset=entities, target_entity="id")
feature_matrix.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,MONTH(precipitationTime),MONTH(gps_datetimeSB),WEEKDAY(alertDateTimeSB),WEEKDAY(gps_datetime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime),WEEKDAY(precipitationTimeSB),WEEKDAY(jamUpdateDateTimeSB),WEEKDAY(precipitationTime),WEEKDAY(gps_datetimeSB)
id,,,,,,,,,,,,,,,,,,,,,
723,203,1,1943,high_frequency,4812142,-25.444150,-49.265501,9021.842,BE721,956958.0,...,1,1,2,2,2,2,2,2,2,2
724,203,1,1943,high_frequency,4812076,-25.472440,-49.251755,5592.591,BE721,956915.0,...,1,1,2,2,2,2,2,2,2,2
725,203,1,1943,high_frequency,4812093,-25.466695,-49.254369,6282.288,BE721,NaN,...,1,1,2,2,2,2,2,2,2,2
726,203,1,1943,high_frequency,4812108,-25.463026,-49.256323,6735.640,BE721,NaN,...,1,1,2,2,2,2,2,2,2,2
727,203,1,1943,high_frequency,4812119,-25.458404,-49.258586,7296.021,BE721,956877.0,...,1,1,2,2,2,2,2,2,2,2


In [15]:
# Saving file
output_path = integrated_data_path + "/" + "new_feats_data.csv"
feature_matrix.to_csv(output_path, index=False)

In [14]:
features_defs

[<Feature: route>,
 <Feature: tripNum>,
 <Feature: shapeId>,
 <Feature: routeFrequency>,
 <Feature: shapeSequence>,
 <Feature: shapeLat>,
 <Feature: shapeLon>,
 <Feature: distanceTraveledShape>,
 <Feature: busCode>,
 <Feature: gpsPointId>,
 <Feature: gpsLat>,
 <Feature: gpsLon>,
 <Feature: distanceToShapePoint>,
 <Feature: stopPointId>,
 <Feature: problem>,
 <Feature: precipitation>,
 <Feature: precipitationStationDistance>,
 <Feature: alertSubtype>,
 <Feature: alertType>,
 <Feature: alertRoadType>,
 <Feature: alertConfidence>,
 <Feature: alertNComments>,
 <Feature: alertNImages>,
 <Feature: alertNThumbsUp>,
 <Feature: alertReliability>,
 <Feature: alertReportMood>,
 <Feature: alertReportRating>,
 <Feature: alertSpeed>,
 <Feature: alertLatitude>,
 <Feature: alertLongitude>,
 <Feature: alertDistanceToClosestShapePoint>,
 <Feature: alertIsJamUnifiedAlert>,
 <Feature: alertInScale>,
 <Feature: jamBlockType>,
 <Feature: jamDelay>,
 <Feature: jamLength>,
 <Feature: jamLevel>,
 <Feature: jam

About feature types:
* It works with data type, but generate just mean, sum, count, mode...